In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import io
import dvc.api
import seaborn as sns

In [8]:
data = dvc.api.read(path='data/train_store.csv',repo='https://github.com/MohammedEsamaldin/Pharmaceutical-Sales-Prediction-across-multiple-stores.git')
df = pd.read_csv(io.StringIO(data),low_memory=False)

PathMissingError: The path 'data/train_store.csv' does not exist in the target repository 'https://github.com/MohammedEsamaldin/Pharmaceutical-Sales-Prediction-across-multiple-stores.git' neither as a DVC output nor as a Git-tracked file.

In [3]:
df.head()

,outs:
0,- md5: 4f7cba67f44211adf5824cefdde01cb8
1,size: 83923976
2,path: train_store.csv
